In [1]:
import json
import csv
import requests
import logging
import glob
import os
import pendulum
from dotenv import load_dotenv


def get_authorization_token():
    load_dotenv()
    authorization_token = os.environ.get('authorization_token')

    return authorization_token


def filename():
    csv_file = glob.glob('*.csv')[0]
    return csv_file


def csv_to_json():
    jsonArray = []
    inputs = {}
    webServiceInput0 = {}

    manual_input_file = filename()

    # Encoding is set that option to remove Byte Order Mark in id key
    with open(manual_input_file, encoding='utf-8-sig') as csvfile:
        csvReader = csv.DictReader(csvfile)
        for row in csvReader:
            jsonArray.append(row)

    # Remove the flag bought column
    for i in jsonArray:
            i.pop("flag_bought", None)

    webServiceInput0['WebServiceInput0'] = jsonArray
    inputs['Inputs'] = webServiceInput0
    inputs['GlobalParameters'] = {}

    jsonString = json.dumps(inputs)

    return jsonString


def calculate_score(inputData):
    bearer_token = get_authorization_token()
    url = "https://mymagicnumberi59jz7.southeastasia.cloudapp.azure.com:443/api/v1/service/crystalball-01-lg-r01/score"
    headers = {
        'Content-Type': 'application/json',
        'Authorization':f'{bearer_token}'
        }
    
    # To pass the JSON payload to endpoint for probability score
    response = requests.request("POST", url, headers=headers, data=inputData)
    json_data = json.loads(response.text)
    

    logging.info("Probability score is calculated")

    return json_data


def write_json_file(probability_score):
    current_date = pendulum.today('Asia/Kuala_Lumpur')
    current_date = current_date.format('DDMMYYYY')

    with open(f"{current_date}_manual_output.json", "w") as jsonfile:
        json.dump(probability_score, jsonfile)

    logging.info("JSON File is created")



if __name__ == "__main__":
    logging.basicConfig(level = logging.INFO)

    try:
        ml_payload = csv_to_json()
        print(ml_payload)
        json_ml_payload = json.loads(ml_payload)

        if len(json_ml_payload['Inputs']['WebServiceInput0']) > 0:
            probability_score_payload = calculate_score(ml_payload)
            write_json_file(probability_score_payload)

        else:
            logging.info("No input data from stored procedure")

    except Exception as e:
        logging.info(f"{e}")

{"Inputs": {"WebServiceInput0": [{"id": "1301", "is_oku": "0", "bumiputra_status": "bumiputra", "customer_type": "earlybird", "gender": "Female", "age": "35.4384", "Age_Missing_Flag": "0", "count_login": "0", "count_preview_project": "2", "count_preview_unit": "21", "count_preview_unit_range": "Nov-50", "count_creditcheck_complete": "0", "count_creditcheck_results": "0", "phase_id": "82", "township_id": "3", "gross_salary": "NULL", "b_form_revenue": "NULL", "commission": "NULL", "total_income": "0", "total_income_missing_flag": "0", "applicant_rating_dsr": "NULL", "applicant_rating_dsr_missing_flag": "1", "applicant_rating_dsr_median": "38.77", "applicant_rating_ccris": "Unspecified", "applicant_rating_ctos": "Unspecified", "applicant_rating_overall": "Unspecified", "employment_type": "Unspecified", "employer_type": "Unspecified", "applicant_nationality": "Unspecified", "income_type": "Unspecified", "no_of_existing_prop_loan": "0", "no_of_existing_prop_loan_missing_flag": "1", "dob": "

INFO:root:Probability score is calculated
INFO:root:JSON File is created
